In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown, stopwords
import itertools
from collections import Counter
import json

nltk.download('brown')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = stopwords.words('english')

np.set_printoptions(suppress=True)

[nltk_data] Downloading package brown to /Users/r0g06z5/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/r0g06z5/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/r0g06z5/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
ambiguous_word = 'apple'
word_senses = wn.synsets(ambiguous_word)
print('Number of senses:', len(word_senses))

Number of senses: 2


In [3]:
def get_sense(sample_sentence, ambiguous_word, feature):
    words = sample_sentence.split(' ')
    words = [w.lower() for w in words]
    words.remove(ambiguous_word)

    sense_bag = []
    context_bag = []
    for word in words:
        word_senses = wn.synsets(word)

        if feature == 'hyponym':
            word_hyponyms = list(itertools.chain(*map(lambda x: x.hyponyms(), word_senses)))
            context_bag += word_hyponyms
        if feature == 'hypernym':
            word_hypernyms = list(itertools.chain(*map(lambda x: x.hypernyms(), word_senses)))
            context_bag += word_hypernyms

    ambiguous_word_senses = wn.synsets(ambiguous_word)
    if feature=='hyponym':
        sense_bag = list(map(lambda x: x.hyponyms(), ambiguous_word_senses))
    if feature=='hypernym':
        sense_bag = list(map(lambda x: x.hypernyms(), ambiguous_word_senses))
    return sense_bag, set(context_bag)  

In [4]:
sample_sentence = 'I am eating apple'
ambiguous_word = 'apple'
feature = 'hyponym'

sense_bag, context_bag = get_sense(sample_sentence, ambiguous_word, feature)
count_overlap = list(map(lambda x: len(set(x) and context_bag), sense_bag))
sense_idx = count_overlap.index(max(count_overlap))
wn.synsets(ambiguous_word)[sense_idx].hypernyms()

[Synset('edible_fruit.n.01'), Synset('pome.n.01')]

In [5]:
def preprocess(sense):
    words = sense.definition().split()
    words = [word.lower() for word in words if word.isalnum() and word not in stopwords]
    return words

context_bag = set(itertools.chain(*map(lambda x: preprocess(x), wn.synsets('coal'))))

In [6]:
sense_bag = list(map(lambda x: preprocess(x), wn.synsets('ash')))
count_overlap = list(map(lambda x: len(set(x) and context_bag), sense_bag))
sense_idx = count_overlap.index(max(count_overlap))
sense_idx

0